# Chapter 5. backPropagation (오차역전파법)

In [1]:
import numpy as np

In [2]:
# Multiply Layer
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dOut):
        dx = dOut * self.y  # change x and y
        dy = dOut * self.x
        
        return dx, dy
    
# Add Layer
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dOut):
        dx = dOut * 1
        dy = dOut * 1
        return dx, dy

In [3]:
# apply each Layers
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layers
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dPrice = 1
dAll_price, dTax = mul_tax_layer.backward(dPrice)
dApple_price, dOrange_price = add_apple_orange_layer.backward(dAll_price)
dOrange, dOrange_num = mul_orange_layer.backward(dOrange_price)
dApple, dApple_num = mul_apple_layer.backward(dApple_price)

print(price)
print(dApple_num, dApple, dOrange, dOrange_num, dTax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [4]:
# Relu class
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dOut):
        dOut[self.mask] = 0
        dx = dOut
        
        return dx

# Sigmoid class
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dOut):
        dx = dOut * (1.0 - self.out) * self.out
        
        return dx

In [5]:
# Affine class (Affine Transformation : scalar product implemented in forward calculation in NN)
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, slef.W) + self.b
        
        return out
    
    def backward(self, dOut):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dOut)
        self.db = np.sum(dOut, axis=0)
        
        return dx
    
# Softmax-with-Loss class (used in printing layer)
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # loss
        self.y = None     # softmax의 출력
        self.t = None      # answer label (one-hot vector)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dOut=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

## Overall Procedure of Neural Network Training

4 steps in training NN
* 1st - mini-batch
* 2nd - calculate gradient
* 3rd - update parameters
* 4th - iterate

In [7]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict     # we save layer of Neural Network in OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # initiallize weights
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # generate classes
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        slef.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    # x: input data, t: answer label
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = p.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x: input data, t: answer label
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dOut = 1
        dOut = self.lastLayer.backward(dOut)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dOut = layer.backward(dOut)
            
        # save results
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

## Evaluate Gradients Calculated by BackPropagation

We use Numerical Differentiaion to evaluate our Back Propagation model.

In [11]:
# gradient check
import sys, os
#sys.setdefaultencoding("utf-f")
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# read data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# average of absolute loss of each weights
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

## Training Neural Network using Back Propagation

The only difference is that we calculate gradients using back propagation

In [10]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

# read data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

# hyperparameters
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

for i in range(iters_num):
    # mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # calculate gradient using Back Propagation
    # grad = network.numerical_gradient(x_batch, t_batch)  # nope
    grad = network.gradient(x_batch, t_batch) # better version = backPropagation
    
    # update parameters
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # record as training passed
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # calculate accuracy per 1 epoch
    if 1 % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xb9 in position 203: ordinal not in range(128)